In [2]:
import pandas as pd
pd.options.display.max_columns = 50

xls = pd.ExcelFile(r'C:\Users\User\Desktop\PROJEKTY_VS\ICY24-CapeTown\transcriptomics20240424\All Results.xlsx')

df = pd.read_excel(xls)
df.drop(['OD', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'FC-OD', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
         'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'FC-OD.1', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
         'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'mean FC-OD', 'stat OD', 'Unnamed: 53', 'mean FC-OD.1',
         'stat OD.1', 'Unnamed: 56', 'mean FC-FL', 'stat FL', 'Unnamed: 59', 'mean FC-sFL', 'stat sFL', 'FC-FL', 'Unnamed: 36',
         'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'FC-sFL', 'Unnamed: 44',
         'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50'], axis=1, inplace=True)

df = df[~df['Unnamed: 2'].isin([3.0, 7.0])]
df = df[~df['Unnamed: 3'].isin([22.0, 34.0])]
df = df[df['Unnamed: 4'] == 'Y']
df = df[df['Unnamed: 5'] == 'Glu']
df = df[~df['TF'].isin([2900])]
df = df[~df['variant'].isin([9.2, 9.1, 11.1, 11.2, 10.1, 10.2])]
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)

print(df.head())

      TF  variant  Unnamed: 2  Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6  \
0   2810     12.1         5.0        28.0          Y        Glu         AS   
1   2810     12.2         5.0        28.0          Y        Glu         CH   
2   2810     13.1         5.0        28.0          Y        Glu         AS   
3   2810     13.2         5.0        28.0          Y        Glu         CH   
4  TF001     12.1         5.0        28.0          Y        Glu         AS   

        FL  Unnamed: 12  Unnamed: 13  Unnamed: 14          sFL  Unnamed: 16  \
0  24739.0      24687.0      23673.0      22568.0  1917.766804  1900.330231   
1  23842.0      23452.0      27547.0      32656.0  1434.830230  1441.239914   
2  21255.0      22997.0      25714.0      24635.0  1593.722585  1763.533048   
3  23283.0      24219.0      24739.0      25532.0  1353.332326  1421.052632   
4  25610.0      25701.0          NaN          NaN  1956.694478  2018.582806   

   Unnamed: 17  Unnamed: 18  
0  1805.831019  1716.43266

In [3]:
df.drop(['variant', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'], axis=1, inplace=True)

df_rawFL = df[['TF', 'FL', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14']]
df_rawFL = df_rawFL.melt(id_vars='TF', value_vars= ['FL', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14'], value_name='raw_FL')
df_rawFL.drop(['variable'], axis=1, inplace=True)


In [4]:
df_rawFL['TF'] = df_rawFL['TF'].astype('str')
df_rawFL.sort_values(by=['TF'], axis=0, inplace=True, ignore_index=True)
df_rawFL.dropna(inplace=True)
df_rawFL.reset_index(inplace=True)
df_rawFL.drop(['index'], axis=1, inplace=True)

print(df_rawFL.head(30))

       TF   raw_FL
0    2810  24739.0
1    2810  25714.0
2    2810  24739.0
3    2810  24219.0
4    2810  22997.0
5    2810  23452.0
6    2810  24687.0
7    2810  27547.0
8    2810  22568.0
9    2810  32656.0
10   2810  24635.0
11   2810  25532.0
12   2810  23673.0
13   2810  23842.0
14   2810  21255.0
15   2810  23283.0
16  TF001  22178.0
17  TF001  19890.0
18  TF001  22763.0
19  TF001  25610.0
20  TF001  25701.0
21  TF001  21983.0
22  TF001  21268.0
23  TF001  22113.0
24  TF002  21229.0
25  TF002  24570.0
26  TF002  25857.0
27  TF002  18837.0
28  TF002  17238.0
29  TF002  18239.0


In [5]:
#%pip install statsmodels 
#%pip install scipy

import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd



In [6]:
# Perform ANOVA for raw FL
model = ols('raw_FL ~ C(TF)', data=df_rawFL).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("ANOVA Table:")
print(anova_table)

ANOVA Table:
                sum_sq     df         F        PR(>F)
C(TF)     2.725556e+10  124.0  6.992291  5.130019e-71
Residual  2.775713e+10  883.0       NaN           NaN


In [7]:
# Perform post-hoc analysis for raw FL
tukey = pairwise_tukeyhsd(endog=df_rawFL['raw_FL'], groups=df_rawFL['TF'], alpha=0.05)
print("Tukey's HSD results:")
print(tukey)


Tukey's HSD results:
     Multiple Comparison of Means - Tukey HSD, FWER=0.05      
group1 group2  meandiff  p-adj     lower       upper    reject
--------------------------------------------------------------
  2810  TF001  -2032.875    1.0 -12757.8885   8692.1385  False
  2810  TF002  -2907.125    1.0 -13632.1385   7817.8885  False
  2810  TF003  11061.375 0.0291    336.3615  21786.3885   True
  2810  TF004   -931.125    1.0 -11656.1385   9793.8885  False
  2810  TF005   1672.125    1.0  -9052.8885  12397.1385  False
  2810  TF006   -2603.25    1.0 -13328.2635   8121.7635  False
  2810  TF007   -2609.75    1.0 -13334.7635   8115.2635  False
  2810  TF009   -1400.75    1.0 -12125.7635   9324.2635  False
  2810  TF010  -4285.125    1.0 -15010.1385   6439.8885  False
  2810  TF011   -1966.25    1.0 -12691.2635   8758.7635  False
  2810  TF012    -965.25    1.0 -11690.2635   9759.7635  False
  2810  TF013   -1153.75    1.0 -11878.7635   9571.2635  False
  2810  TF014   -1244.75    1.0 -1

In [11]:
df_normFL = df[['TF', 'sFL', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']]
df_normFL = df_normFL.melt(id_vars='TF', value_vars= ['sFL', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'], value_name='normalized_FL')
df_normFL.drop(['variable'], axis=1, inplace=True)

df_normFL['TF'] = df_normFL['TF'].astype('str')
df_normFL.sort_values(by=['TF'], axis=0, inplace=True, ignore_index=True)
df_normFL.dropna(inplace=True)
df_normFL.reset_index(inplace=True)
df_normFL.drop(['index'], axis=1, inplace=True)

print(df_normFL.head(30))

       TF  normalized_FL
0    2810    1917.766804
1    2810    1779.417056
2    2810    1478.287889
3    2810    1421.052632
4    2810    1763.533048
5    2810    1441.239914
6    2810    1900.330231
7    2810    1595.152063
8    2810    1716.432668
9    2810    1902.597894
10   2810    1779.676935
11   2810    1546.456693
12   2810    1805.831019
13   2810    1434.830230
14   2810    1593.722585
15   2810    1353.332326
16  TF001    1312.105830
17  TF001    1548.426273
18  TF001    1345.267363
19  TF001    1956.694478
20  TF001    2018.582806
21  TF001    1706.701655
22  TF001    1293.689704
23  TF001    1328.594861
24  TF002    1286.840032
25  TF002    1457.546079
26  TF002    2067.997505
27  TF002    1161.895598
28  TF002    1442.715700
29  TF002    1457.662338


In [12]:
# Perform ANOVA for normalized FL
model = ols('normalized_FL ~ C(TF)', data=df_normFL).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("ANOVA Table:")
print(anova_table)

ANOVA Table:
                sum_sq     df         F        PR(>F)
C(TF)     9.786018e+07  124.0  6.183367  2.205582e-61
Residual  1.126990e+08  883.0       NaN           NaN


In [13]:
# Perform post-hoc analysis for normalized FL
tukey = pairwise_tukeyhsd(endog=df_normFL['normalized_FL'], groups=df_normFL['TF'], alpha=0.05)
print("Tukey's HSD results:")
print(tukey)


Tukey's HSD results:
    Multiple Comparison of Means - Tukey HSD, FWER=0.05     
group1 group2  meandiff  p-adj    lower      upper    reject
------------------------------------------------------------
  2810  TF001   -88.0959    1.0  -771.4889   595.2972  False
  2810  TF002  -108.7501    1.0  -792.1431    574.643  False
  2810  TF003   524.5655 0.7225  -158.8276  1207.9585  False
  2810  TF004   -28.9973    1.0  -712.3903   654.3958  False
  2810  TF005   188.8032    1.0  -494.5898   872.1963  False
  2810  TF006  -141.0412    1.0  -824.4342   542.3519  False
  2810  TF007  -105.9266    1.0  -789.3196   577.4665  False
  2810  TF009   -47.0744    1.0  -730.4674   636.3187  False
  2810  TF010  -220.0515    1.0  -903.4445   463.3416  False
  2810  TF011  -103.0878    1.0  -786.4809   580.3052  False
  2810  TF012   -74.9649    1.0  -758.3579   608.4282  False
  2810  TF013   -73.6397    1.0  -757.0328   609.7533  False
  2810  TF014   -83.3868    1.0  -766.7798   600.0063  False
  2